<img src="https://avatars.githubusercontent.com/u/74911464?s=200&v=4"
     alt="OpenEO Platform logo"
     style="float: left; margin-right: 10px;" />
# openEO Platform UC8
## Random Forest Regression - Fractional Canopy Cover prediction

### Requirements

In [1]:
import openeo
from openeo.processes import median, mean
from openeo.rest.datacube import PGNode, THIS
import geopandas as gpd

## Connection

Connect to the OpenEO back-end

In [2]:
# conn = openeo.connect("openeo.cloud")
conn = openeo.connect("https://openeo-dev.eodc.eu/v1.0")

Authenticate via EGI Check-in (OpenID Connect)

In [3]:
conn.authenticate_oidc("egi")

Authenticated using refresh token.


<Connection to 'https://openeo-dev.eodc.eu/v1.0' with BearerAuth>

In the previous notebook, we have trained a Random Forest Regression model with the Fractional Canopy Cover data. Now it's time to predict the values over a large area and inspect the result.

The spatial extent must be within the boundaries of the `boa_sentinel_2` and `SIG0_Sentinel_1` collections:

In [5]:
spatial_extent  =  {"west":10.454955,
                    "east":12.037297,
                    "south":46.102185,
                    "north":47.053657}

Load the Sentinel-2 data for summer 2018 and take the median over time:

In [6]:
collection      = "boa_sentinel_2"
temporal_extent = ["2018-05-01", "2018-09-01"]
bands           = ["B02","B03","B04","B08"]

boa_sentinel_2_cube = conn.load_collection(
    collection_id   = collection,
    spatial_extent  = spatial_extent,
    temporal_extent = temporal_extent,
    bands = bands
    )
boa_sentinel_2_cube_reduced = boa_sentinel_2_cube.reduce_dimension(dimension="t",reducer=median)

Load the Sentinel-1 data for summer 2018 and take the median over time:

In [7]:
collection      = "SIG0_Sentinel_1"
temporal_extent = ["2018-05-01", "2018-09-01"]
bands           = ["vh","vv"]

sigma0_sentinel_1_cube = conn.load_collection(
    collection_id   = collection,
    spatial_extent  = spatial_extent,
    temporal_extent = temporal_extent,
    bands = bands
    )
sigma0_sentinel_1_cube_reduced = sigma0_sentinel_1_cube.reduce_dimension(dimension="t",reducer=median)

Reproject Sentinel-1 20m to match the Sentinel-2 data in 10m using `resample_cube_spatial` and nearest neighbor interpolation (default):

In [8]:
sigma0_sentinel_1_cube_reduced_10m = sigma0_sentinel_1_cube_reduced.resample_cube_spatial(target=boa_sentinel_2_cube_reduced)

Merge Sentinel-2 and Sentinel-1 into a single datacube

In [9]:
sentinel_2_and_1 = sigma0_sentinel_1_cube_reduced_10m.merge_cubes(boa_sentinel_2_cube_reduced)

Flatten the x and y dimension to get a 2 dimensional datacube to feed into the Random Forest.

In [10]:
sentinel_2_and_1_flat = sentinel_2_and_1.process("flatten_dimensions",{"data":THIS,"dimensions":["y","x"], "target_dimension":"result"})

We start loading our previously generated model. We can use the job id of the correspondent batch job or the URL of the result .json file with the model.

In [ ]:
model_job_id = ""
rf_regr_model = PGNode("load_ml_model", {"model": model_job_id})

Feed the flattened data and the model in `predict_random_forest` witin a `reduce_dimension` (we are reducing the bands to a single one, the fractional canopy cover)

In [17]:
fractional_canopy_cover_flat = sentinel_2_and_1_flat.process("predict_random_forest",{"data":THIS, "dimension":"bands", "model":rf_regr_model})

Get the data back to its original shape in x and y dimensions:

In [18]:
fractional_canopy_cover = fractional_canopy_cover_flat.process("unflatten_dimension",{"data":THIS,"dimensions":"result", "target_dimension":["y","x"]})

In [13]:
fractional_canopy_cover_netcdf = fractional_canopy_cover.save_result(format="netCDF")

In [ ]:
job = fractional_canopy_cover_netcdf.send_job(title="UC8_predict_rf_regr")
job_id = job.job_id
if job_id:
    print("Batch job created with id: ",job_id)
    job.start_job()
else:
    print("Error! Job ID is None")